In [1]:
import numpy as np 
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn import preprocessing

train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=['startdate'])
test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=['startdate'])
columns = train.columns[train.isna().any()].tolist()
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)
print(columns)
label = "contest-tmp2m-14d__tmp2m"
climate_regions = 'climateregions__climateregion'

['nmme0-tmp2m-34w__ccsm30', 'nmme-tmp2m-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme0-prate-56w__ccsm30', 'nmme0-prate-34w__ccsm30', 'nmme-prate-56w__ccsm3', 'nmme-tmp2m-34w__ccsm3', 'ccsm30']


In [2]:
def location_norm(train, test):
        scale = 14
        train.loc[:,'lat'] = round(train.lat, scale)
        train.loc[:,'lon'] = round(train.lon, scale)
        test.loc[:,'lat'] = round(test.lat, scale)
        test.loc[:,'lon'] = round(test.lon, scale)
        return train, test
    
def preprocess(train, test):
    train, test = location_norm(train, test)
    features = ['wind-vwnd-250-2010-9', 'wind-hgt-100-2010-4',
       'wind-vwnd-925-2010-10', 'wind-uwnd-250-2010-17',
       'wind-hgt-850-2010-9', 'wind-uwnd-925-2010-10',
       'wind-vwnd-925-2010-12', 'wind-vwnd-925-2010-19',
       'wind-uwnd-925-2010-20', 'wind-uwnd-925-2010-12',
       'wind-hgt-100-2010-10', 'wind-vwnd-925-2010-15',
       'wind-vwnd-250-2010-8', 'wind-uwnd-925-2010-9',
       'wind-uwnd-925-2010-5', 'wind-uwnd-250-2010-5',
       'wind-hgt-500-2010-9', 'wind-hgt-500-2010-7',
       'wind-vwnd-925-2010-8', 'wind-vwnd-925-2010-13',
       'wind-hgt-500-2010-10', 'wind-hgt-500-2010-6',
       'wind-hgt-500-2010-2', 'wind-uwnd-925-2010-15',
       'wind-vwnd-250-2010-16', 'wind-hgt-10-2010-6',
       'wind-vwnd-250-2010-4', 'wind-vwnd-925-2010-18',
       'wind-uwnd-925-2010-18', 'wind-hgt-10-2010-3',
       'wind-hgt-500-2010-4', 'wind-uwnd-250-2010-9', 'mjo1d__amplitude',
       'wind-uwnd-250-2010-10', 'wind-uwnd-925-2010-11',
       'wind-vwnd-250-2010-5', 'wind-vwnd-925-2010-16',
       'wind-hgt-10-2010-7', 'wind-hgt-850-2010-7',
       'wind-vwnd-925-2010-14', 'wind-hgt-100-2010-8',
       'wind-vwnd-250-2010-18', 'sst-2010-8', 'sst-2010-7',
       'wind-uwnd-250-2010-14', 'wind-vwnd-250-2010-13',
       'wind-uwnd-925-2010-8', 'wind-hgt-500-2010-3',
       'wind-hgt-10-2010-5', 'wind-uwnd-250-2010-7', 'sst-2010-6',
       'wind-uwnd-925-2010-13', 'wind-vwnd-250-2010-3', 'sst-2010-10',
       'wind-vwnd-925-2010-3', 'wind-uwnd-925-2010-19',
       'wind-uwnd-250-2010-11', 'wind-vwnd-250-2010-10',
       'wind-uwnd-250-2010-15', 'sst-2010-9', 'mei__mei',
       'wind-uwnd-925-2010-14', 'wind-vwnd-250-2010-11',
       'contest-wind-vwnd-250-14d__wind-vwnd-250', 'wind-hgt-100-2010-9',
       'wind-uwnd-925-2010-3', 'wind-vwnd-925-2010-7',
       'contest-wind-h850-14d__wind-hgt-850',
       'contest-wind-uwnd-250-14d__wind-uwnd-250',
       'contest-wind-uwnd-925-14d__wind-uwnd-925',
       'contest-wind-vwnd-925-14d__wind-vwnd-925',
       'contest-precip-14d__precip', 'wind-uwnd-250-2010-4',
       'wind-uwnd-250-2010-2', 'contest-rhum-sig995-14d__rhum',
       'contest-wind-h10-14d__wind-hgt-10', 'wind-uwnd-250-2010-3',
       'wind-hgt-500-2010-1', 'wind-hgt-100-2010-2',
       'contest-pres-sfc-gauss-14d__pres', 'elevation__elevation',
       'contest-slp-14d__slp', 'lon', 'lat',
       'contest-prwtr-eatm-14d__prwtr',
       'contest-wind-h100-14d__wind-hgt-100',
       'contest-wind-h500-14d__wind-hgt-500',
       'contest-pevpr-sfc-gauss-14d__pevpr']
    
    x_train = train[features]
    y_train = train[label]
    x_test = test[features] 
    
    return x_train, y_train, x_test

X, Y, X_test = preprocess(train, test)
X

,wind-vwnd-250-2010-9,wind-hgt-100-2010-4,wind-vwnd-925-2010-10,wind-uwnd-250-2010-17,wind-hgt-850-2010-9,wind-uwnd-925-2010-10,wind-vwnd-925-2010-12,wind-vwnd-925-2010-19,wind-uwnd-925-2010-20,wind-uwnd-925-2010-12,...,wind-hgt-100-2010-2,contest-pres-sfc-gauss-14d__pres,elevation__elevation,contest-slp-14d__slp,lon,lat,contest-prwtr-eatm-14d__prwtr,contest-wind-h100-14d__wind-hgt-100,contest-wind-h500-14d__wind-hgt-500,contest-pevpr-sfc-gauss-14d__pevpr
0,15.14,3391.32,69.09,-71.99,-160.02,17.96,-37.21,11.90,39.02,66.65,...,-5160.59,98644.97,200.0,101352.08,0.833333,0.0,42.45,16666.81,5899.66,237.00
1,-2.39,3188.99,71.93,-62.79,-71.80,29.89,-36.57,15.44,36.31,61.97,...,-5356.70,98686.80,200.0,101396.02,0.833333,0.0,42.66,16667.31,5901.03,228.90
2,-20.02,2996.82,68.09,-52.32,39.03,43.62,-34.16,15.11,32.81,56.82,...,-5546.88,98712.85,200.0,101429.25,0.833333,0.0,43.23,16668.39,5902.18,220.69
3,-37.61,2799.89,60.20,-45.98,206.58,57.48,-31.04,18.24,27.68,49.49,...,-5692.21,98711.70,200.0,101440.85,0.833333,0.0,43.11,16667.39,5903.07,225.28
4,-53.54,2582.56,52.09,-39.14,427.52,69.27,-31.80,21.91,19.15,40.16,...,-5754.12,98686.46,200.0,101419.53,0.833333,0.0,42.98,16665.65,5903.36,237.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,-15.20,7249.39,-3.08,-51.36,-840.25,36.06,-75.68,-31.33,-60.12,22.51,...,-3558.83,97613.96,100.0,101373.91,0.866667,1.0,24.32,16540.22,5763.23,312.05
375730,-7.43,7170.60,-7.08,-45.39,-695.27,43.59,-76.42,-30.73,-56.17,23.93,...,-4272.66,97631.29,100.0,101397.77,0.866667,1.0,23.92,16534.71,5760.19,305.82
375731,8.96,7110.42,-10.98,-53.33,-550.87,43.56,-70.65,-18.74,-45.84,29.32,...,-4872.78,97588.69,100.0,101368.67,0.866667,1.0,23.94,16526.98,5754.76,311.62
375732,22.47,7033.49,-11.03,-70.38,-469.71,44.50,-57.67,-16.26,-35.76,32.90,...,-5406.42,97538.62,100.0,101321.24,0.866667,1.0,23.61,16512.39,5742.21,304.54


In [4]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

folds = 5
seed = 2018
kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
kf.get_n_splits(X)
result = np.empty((folds,X_test.shape[0]))
cv = []
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train = X.iloc[train_index]
    X_val = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_val = Y.iloc[test_index]
    
    opt_params = {
      'iterations': 24000,
      'verbose':0,
      'learning_rate' : 0.0980689972639084,
      'subsample' : 0.7443133148363695, 
      'l2_leaf_reg' : 2.3722386345448316,
      'max_depth' : int(6.599144674342465),
      'use_best_model' : True, 
      'loss_function' : 'RMSE',
      'model_size_reg' : 0.4833187897595954,
     }
    model = CatBoostRegressor(**opt_params)
    train_pool = Pool(data=X_train, label=Y_train)
    model.fit(train_pool, eval_set=(X_val, Y_val), plot=True, verbose=1000)
    cv.append(mean_squared_error(Y_val.values, model.predict(X_val)))
    result[i,:] =  model.predict(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.0563572	test: 9.0791023	best: 9.0791023 (0)	total: 193ms	remaining: 1h 17m 15s
1000:	learn: 0.6255743	test: 0.6409066	best: 0.6409066 (1000)	total: 1m 41s	remaining: 38m 47s
2000:	learn: 0.4741957	test: 0.4956458	best: 0.4956458 (2000)	total: 3m 21s	remaining: 37m
3000:	learn: 0.4001345	test: 0.4270985	best: 0.4270985 (3000)	total: 5m 1s	remaining: 35m 6s
4000:	learn: 0.3521948	test: 0.3829978	best: 0.3829978 (4000)	total: 6m 42s	remaining: 33m 29s
5000:	learn: 0.3173963	test: 0.3518975	best: 0.3518975 (5000)	total: 8m 25s	remaining: 32m 2s
6000:	learn: 0.2913419	test: 0.3292961	best: 0.3292961 (6000)	total: 10m 11s	remaining: 30m 33s
7000:	learn: 0.2703256	test: 0.3111953	best: 0.3111953 (7000)	total: 11m 59s	remaining: 29m 7s
8000:	learn: 0.2526512	test: 0.2960575	best: 0.2960575 (8000)	total: 13m 51s	remaining: 27m 41s
9000:	learn: 0.2382871	test: 0.2843130	best: 0.2843130 (9000)	total: 15m 45s	remaining: 26m 15s
10000:	learn: 0.2259292	test: 0.2742859	best: 0.2742859 (1

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.0620864	test: 9.0562980	best: 9.0562980 (0)	total: 126ms	remaining: 50m 12s
1000:	learn: 0.6229615	test: 0.6350086	best: 0.6350086 (1000)	total: 1m 36s	remaining: 36m 46s
2000:	learn: 0.4744883	test: 0.4940320	best: 0.4940320 (2000)	total: 3m 15s	remaining: 35m 46s
3000:	learn: 0.3987234	test: 0.4239274	best: 0.4239274 (3000)	total: 4m 56s	remaining: 34m 32s
4000:	learn: 0.3512227	test: 0.3816011	best: 0.3816011 (4000)	total: 6m 39s	remaining: 33m 18s
5000:	learn: 0.3169504	test: 0.3515806	best: 0.3515806 (5000)	total: 8m 25s	remaining: 31m 58s
6000:	learn: 0.2904399	test: 0.3283745	best: 0.3283745 (6000)	total: 10m 12s	remaining: 30m 35s
7000:	learn: 0.2692047	test: 0.3102863	best: 0.3102863 (7000)	total: 12m 2s	remaining: 29m 14s
8000:	learn: 0.2522950	test: 0.2960158	best: 0.2960158 (8000)	total: 13m 56s	remaining: 27m 51s
9000:	learn: 0.2377671	test: 0.2841298	best: 0.2841298 (9000)	total: 15m 51s	remaining: 26m 24s
10000:	learn: 0.2255152	test: 0.2743210	best: 0.274321

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.0690110	test: 9.0270916	best: 9.0270916 (0)	total: 127ms	remaining: 50m 35s
1000:	learn: 0.6271370	test: 0.6440673	best: 0.6440673 (1000)	total: 1m 37s	remaining: 37m 10s
2000:	learn: 0.4760822	test: 0.4997744	best: 0.4997744 (2000)	total: 3m 15s	remaining: 35m 49s
3000:	learn: 0.3996549	test: 0.4283322	best: 0.4283322 (3000)	total: 4m 56s	remaining: 34m 33s
4000:	learn: 0.3519064	test: 0.3845213	best: 0.3845213 (4000)	total: 6m 40s	remaining: 33m 23s
5000:	learn: 0.3176509	test: 0.3539219	best: 0.3539219 (5000)	total: 8m 26s	remaining: 32m 5s
6000:	learn: 0.2917007	test: 0.3310443	best: 0.3310443 (6000)	total: 10m 15s	remaining: 30m 45s
7000:	learn: 0.2702960	test: 0.3125838	best: 0.3125838 (7000)	total: 12m 7s	remaining: 29m 25s
8000:	learn: 0.2528981	test: 0.2978376	best: 0.2978376 (8000)	total: 14m 2s	remaining: 28m 4s
9000:	learn: 0.2384455	test: 0.2856912	best: 0.2856912 (9000)	total: 15m 58s	remaining: 26m 36s
10000:	learn: 0.2262493	test: 0.2755742	best: 0.2755742 (

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.0566225	test: 9.0806357	best: 9.0806357 (0)	total: 126ms	remaining: 50m 11s
1000:	learn: 0.6260979	test: 0.6429330	best: 0.6429330 (1000)	total: 1m 37s	remaining: 37m 19s
2000:	learn: 0.4762076	test: 0.4996466	best: 0.4996466 (2000)	total: 3m 17s	remaining: 36m 6s
3000:	learn: 0.3996820	test: 0.4276291	best: 0.4276291 (3000)	total: 4m 58s	remaining: 34m 50s
4000:	learn: 0.3506433	test: 0.3822495	best: 0.3822495 (4000)	total: 6m 43s	remaining: 33m 36s
5000:	learn: 0.3162326	test: 0.3514351	best: 0.3514351 (5000)	total: 8m 31s	remaining: 32m 22s
6000:	learn: 0.2903507	test: 0.3283972	best: 0.3283972 (6000)	total: 10m 21s	remaining: 31m 3s
7000:	learn: 0.2697268	test: 0.3106108	best: 0.3106108 (7000)	total: 12m 13s	remaining: 29m 40s
8000:	learn: 0.2525036	test: 0.2960304	best: 0.2960304 (8000)	total: 14m 7s	remaining: 28m 15s
9000:	learn: 0.2377222	test: 0.2835632	best: 0.2835632 (9000)	total: 16m 5s	remaining: 26m 49s
10000:	learn: 0.2255393	test: 0.2735853	best: 0.2735853 (

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.0608346	test: 9.0626195	best: 9.0626195 (0)	total: 126ms	remaining: 50m 27s
1000:	learn: 0.6254192	test: 0.6387994	best: 0.6387994 (1000)	total: 1m 38s	remaining: 37m 38s
2000:	learn: 0.4752387	test: 0.4971145	best: 0.4971145 (2000)	total: 3m 17s	remaining: 36m 13s
3000:	learn: 0.4003293	test: 0.4278105	best: 0.4278105 (3000)	total: 4m 59s	remaining: 34m 53s
4000:	learn: 0.3510973	test: 0.3831773	best: 0.3831773 (4000)	total: 6m 43s	remaining: 33m 39s
5000:	learn: 0.3178408	test: 0.3532595	best: 0.3532595 (5000)	total: 8m 30s	remaining: 32m 19s
6000:	learn: 0.2918977	test: 0.3303316	best: 0.3303316 (6000)	total: 10m 20s	remaining: 31m 1s
7000:	learn: 0.2706257	test: 0.3119447	best: 0.3119447 (7000)	total: 12m 14s	remaining: 29m 43s
8000:	learn: 0.2535772	test: 0.2974272	best: 0.2974272 (8000)	total: 14m 10s	remaining: 28m 20s
9000:	learn: 0.2388570	test: 0.2850585	best: 0.2850585 (9000)	total: 16m 9s	remaining: 26m 54s
10000:	learn: 0.2263905	test: 0.2748048	best: 0.2748048

In [5]:
cv

[0.04356545451214747,
 0.044239931358277765,
 0.04385826179736264,
 0.0435496071051762,
 0.04360419418186036]

In [6]:
duc_tran=np.zeros((X_test.shape[0]))
duc_tran[:]=result.mean(axis=0)
duc_tran

array([29.53139912, 29.54325072, 29.40521799, ...,  6.19137222,
        6.85172962,  7.16945637])

In [7]:
output = pd.DataFrame({
    label : duc_tran,
    "index": test["index"]
})

output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
